In [24]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time


Step 1: Create a dataset object

In [25]:
from torchtext.datasets import IMDB
train_iter, test_iter = IMDB(split=('train', 'test'))

Step 2 Build the data processing pipeline

In [26]:
# https://pytorch.org/text/stable/_modules/torchtext/data/utils.html#get_tokenizer
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('basic_english')

In [27]:
from collections import Counter #ordered dict
from torchtext.vocab import vocab

counter = Counter() 
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = vocab(counter, min_freq=10, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))

In [28]:
print("The length of the new vocab is", len(vocab))
new_stoi = vocab.get_stoi()# Dictionary mapping tokens to indices.
print("The index of '<BOS>' is", new_stoi['<BOS>'])
new_itos = vocab.get_itos() # List mapping indices to tokens.
print("The token at index 2 is", new_itos[2])

The length of the new vocab is 13020
The index of '<BOS>' is 1
The token at index 2 is <EOS>


In [29]:
text_transform = lambda x: [vocab['<BOS>']] + [vocab[token] for token in tokenizer(x)] + [vocab['<EOS>']]
label_transform = lambda x: 1 if x == 'pos' else 0

# Print out the output of text_transform
print("input to the text_transform:", "here is an example")
print("output of the text_transform:", text_transform("here is an example"))

input to the text_transform: here is an example
output of the text_transform: [1, 938, 54, 195, 3244, 2]


Step 3: Generate batch iterator

In [30]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
   label_list, text_list = [], []
   for (_label, _text) in batch:
        label_list.append(label_transform(_label))
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
   return torch.tensor(label_list), pad_sequence(text_list, padding_value=3.0)

train_dataloader = DataLoader(list(train_iter), batch_size=8, shuffle=True, 
                              collate_fn=collate_batch)

Step 4: Iterate batch to train a model

In [31]:
it = iter(train_dataloader)


In [32]:
next(it)

RuntimeError: Token railing not found and default index is not set

In [ ]:
# for idx, (label, text) in enumerate(train_dataloader):
#   model(item)

# Or
next(iter(train_dataloader))